<a href="https://colab.research.google.com/github/azzindani/03_Prompt_Engineering/blob/main/PE_EN_Qwen2_5_1_5B_Med_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importing Modules

In [1]:
!pip install -U -q bitsandbytes
!pip install -q optimum
!pip install -q auto-gptq

In [2]:
import torch

from transformers import (
  AutoTokenizer,
  AutoModelForCausalLM,
  GenerationConfig,
  BitsAndBytesConfig,
  GPTQConfig
)

if torch.cuda.is_available():
  print('GPU is available!')
else:
  print('GPU is not available.')

GPU is available!


## Import Model

In [3]:
model_name = 'Qwen/Qwen2.5-1.5B'

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [5]:
'''bnb_config = BitsAndBytesConfig(
  load_in_4bit = True,
  bnb_4bit_quant_type = 'nf4',
  bnb_4bit_compute_dtype = torch.float16,
  bnb_4bit_use_double_quant = True,
)

model = AutoModelForCausalLM.from_pretrained(
  model_name,
  quantization_config = bnb_config,
  trust_remote_code = True
).to(device) #'''

"bnb_config = BitsAndBytesConfig(\n  load_in_4bit = True,\n  bnb_4bit_quant_type = 'nf4',\n  bnb_4bit_compute_dtype = torch.float16,\n  bnb_4bit_use_double_quant = True,\n)\n\nmodel = AutoModelForCausalLM.from_pretrained(\n  model_name,\n  quantization_config = bnb_config,\n  trust_remote_code = True\n).to(device) #"

In [6]:
model = AutoModelForCausalLM.from_pretrained(
  model_name,
  torch_dtype = torch.float16,
  trust_remote_code = True
).to(device) #'''

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


## Import Tokenizer

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

## Model Preview

In [8]:
def assistant(prompt):
  prompt_format = f"### Human: {prompt}\n### Assistant :"

  inputs = tokenizer(
    prompt_format,
    return_tensors = 'pt'
  ).to(device)
  generation_config = GenerationConfig(
    do_sample = True,
    top_k = 1,
    temperature = 0.1,
    max_new_tokens = 1024,
    pad_token_id = tokenizer.eos_token_id
  )
  outputs = model.generate(
    **inputs,
    generation_config = generation_config
  )
  return print(tokenizer.decode(outputs[0], skip_special_tokens = True))

In [9]:
# Level 1: Basic Prompt
prompt = "What are the symptoms and treatments of diabetes?"
assistant(prompt)

### Human: What are the symptoms and treatments of diabetes?
### Assistant : Diabetes is a chronic disease that affects how your body turns food into energy. It occurs when your blood sugar, or glucose, levels are too high. There are two main types of diabetes: Type 1 and Type 2. Type 1 diabetes is an autoimmune disorder where the body's immune system attacks and destroys the insulin-producing cells in the pancreas. Type 2 diabetes is a metabolic disorder where the body becomes resistant to insulin or doesn't produce enough insulin to function properly. Symptoms of diabetes include increased thirst, frequent urination, fatigue, blurred vision, and slow-healing wounds. Treatment for diabetes involves lifestyle changes such as a healthy diet, regular exercise, and medication. In some cases, insulin injections may be necessary. It's important to manage diabetes to prevent complications such as heart disease, kidney failure, and nerve damage.


In [10]:
# Level 2: Adding Detail
prompt = "Briefly describe the symptoms and treatments of diabetes."
assistant(prompt)

### Human: Briefly describe the symptoms and treatments of diabetes.
### Assistant : Diabetes is a chronic metabolic disorder characterized by high blood sugar levels due to insulin resistance or insufficient insulin production. Symptoms include increased thirst, frequent urination, fatigue, blurred vision, and slow healing wounds. Treatment involves lifestyle changes such as diet and exercise, oral medications like sulfonylureas, and insulin injections for those who cannot produce enough insulin. Regular monitoring of blood sugar levels is crucial for managing the condition.


In [11]:
# Level 3: Specify Information Type and Audience
prompt = "Explain the common symptoms and treatments of diabetes to a general audience."
assistant(prompt)

### Human: Explain the common symptoms and treatments of diabetes to a general audience.
### Assistant : Diabetes is a chronic disease that affects how your body turns food into energy. It occurs when your body doesn't produce enough insulin or can't use the insulin it produces effectively. There are two main types of diabetes: Type 1 and Type 2. Type 1 diabetes is an autoimmune disorder where the body's immune system attacks and destroys the insulin-producing cells in the pancreas. Type 2 diabetes is more common and occurs when the body becomes resistant to insulin or doesn't produce enough insulin. Symptoms of diabetes can include increased thirst, frequent urination, fatigue, blurred vision, and slow-healing wounds. Treatment options for diabetes include lifestyle changes such as diet and exercise, medications, and insulin therapy. It's important to manage diabetes through a combination of these approaches to maintain good health and prevent complications.


In [12]:
# Level 4: Request for Structured Response
prompt = "List the main symptoms of diabetes, then outline the standard treatments."
assistant(prompt)

### Human: List the main symptoms of diabetes, then outline the standard treatments.
### Assistant : Diabetes is a chronic disease that affects how your body turns food into energy. It occurs when your blood sugar, or glucose, levels are too high. There are two main types of diabetes: Type 1 and Type 2. Type 1 diabetes is an autoimmune disorder where the body's immune system attacks and destroys the insulin-producing cells in the pancreas. Type 2 diabetes is a metabolic disorder where the body becomes resistant to insulin or doesn't produce enough insulin to function properly. Symptoms of diabetes include increased thirst, frequent urination, fatigue, blurred vision, and slow-healing wounds. Treatment options for diabetes include lifestyle changes such as diet and exercise, medications, and insulin therapy.


In [13]:
# Level 5: Differentiate by Type
prompt = "List the symptoms and treatments for both Type 1 and Type 2 diabetes."
assistant(prompt)

### Human: List the symptoms and treatments for both Type 1 and Type 2 diabetes.
### Assistant : Type 1 diabetes is an autoimmune disease in which the body's immune system attacks and destroys the insulin-producing beta cells in the pancreas. This leads to a lack of insulin, which is essential for the body to use glucose for energy. Symptoms of Type 1 diabetes include increased thirst, frequent urination, extreme hunger, unexplained weight loss, fatigue, and blurred vision. Treatment for Type 1 diabetes involves insulin therapy, which is administered through injections or an insulin pump. The goal of treatment is to keep blood glucose levels within a healthy range to prevent long-term complications such as kidney damage, nerve damage, and cardiovascular disease. Type 2 diabetes is a chronic condition in which the body becomes resistant to insulin or does not produce enough insulin to regulate blood glucose levels. Symptoms of Type 2 diabetes include increased thirst, frequent urination

In [14]:
# Level 6: Target a Professional Audience
prompt = "Describe the symptoms, diagnostic criteria, and treatment options for Type 1 and Type 2 diabetes, focusing on what healthcare providers should know."
assistant(prompt)

### Human: Describe the symptoms, diagnostic criteria, and treatment options for Type 1 and Type 2 diabetes, focusing on what healthcare providers should know.
### Assistant : Type 1 diabetes is an autoimmune disease in which the body's immune system attacks and destroys the insulin-producing beta cells in the pancreas. This leads to a lack of insulin, which is essential for regulating blood sugar levels. Symptoms of Type 1 diabetes may include increased thirst, frequent urination, extreme hunger, unexplained weight loss, fatigue, and blurred vision. The diagnostic criteria for Type 1 diabetes include a history of symptoms, a positive C-peptide test, and a low or undetectable level of insulin in the blood. Treatment options for Type 1 diabetes include insulin therapy, dietary management, and regular exercise. Healthcare providers should be aware of the potential complications of Type 1 diabetes, such as diabetic ketoacidosis and hyperglycemic hyperosmolar syndrome, and should monitor p

In [15]:
# Level 7: Clinical Detail with Disease Stages
prompt = "Explain the symptoms, diagnostic markers, and treatments for diabetes, distinguishing between prediabetes, Type 1, and Type 2 diabetes. Include early signs, progression, and advanced stages."
assistant(prompt)

### Human: Explain the symptoms, diagnostic markers, and treatments for diabetes, distinguishing between prediabetes, Type 1, and Type 2 diabetes. Include early signs, progression, and advanced stages.
### Assistant : Diabetes is a chronic disease that affects how your body turns food into energy. It occurs when your blood sugar (glucose) levels are too high. There are two main types of diabetes: Type 1 and Type 2. Prediabetes is a condition where your blood sugar levels are higher than normal but not high enough to be diagnosed as diabetes. Early signs of diabetes include increased thirst, frequent urination, fatigue, and blurred vision. As the disease progresses, symptoms may include slow healing of cuts and sores, tingling or numbness in the hands or feet, and slow healing of cuts and sores. Advanced stages of diabetes can lead to complications such as heart disease, kidney failure, and nerve damage. Treatment for diabetes involves lifestyle changes such as a healthy diet, regular e

In [16]:
# Level 8: Add Nuance with Treatment Personalization
prompt = "Describe the symptoms, diagnostic markers, and treatment options for diabetes, with a focus on Type 1 and Type 2. Include how treatments may vary based on patient age, co-existing conditions, and lifestyle factors."
assistant(prompt)

### Human: Describe the symptoms, diagnostic markers, and treatment options for diabetes, with a focus on Type 1 and Type 2. Include how treatments may vary based on patient age, co-existing conditions, and lifestyle factors.
### Assistant : Diabetes is a chronic disease that affects how your body turns food into energy. It can cause high blood sugar levels, which can lead to serious health problems if not managed properly. There are two main types of diabetes: Type 1 and Type 2.

#### Type 1 Diabetes:
- **Symptoms:** Frequent urination, extreme thirst, extreme hunger, unexplained weight loss, fatigue, blurred vision, and slow-healing sores.
- **Diagnostic Markers:** Blood tests to measure blood sugar levels, HbA1c (glycated hemoglobin) test to check average blood sugar levels over the past 2-3 months, and urine tests to check for ketones.
- **Treatment Options:** Insulin therapy, which involves injecting insulin into the body to help control blood sugar levels. Other treatments may in

In [17]:
# Level 9: Consider Risks, Complications, and Treatment Impacts
prompt = "Provide a comprehensive overview of diabetes, covering symptoms, diagnostic criteria, and treatment options for both Type 1 and Type 2 diabetes. Explain how risks and complications, such as neuropathy and cardiovascular disease, may impact treatment decisions. Include possible side effects and patient adherence concerns for each treatment."
assistant(prompt)

### Human: Provide a comprehensive overview of diabetes, covering symptoms, diagnostic criteria, and treatment options for both Type 1 and Type 2 diabetes. Explain how risks and complications, such as neuropathy and cardiovascular disease, may impact treatment decisions. Include possible side effects and patient adherence concerns for each treatment.
### Assistant : Diabetes is a chronic metabolic disorder characterized by high blood sugar levels due to either insufficient insulin production or insulin resistance. It can affect various organs and systems in the body, leading to a range of symptoms and complications.

#### Symptoms of Diabetes:
- Frequent urination
- Increased thirst and hunger
- Fatigue
- Blurred vision
- Slow-healing sores or infections
- Tingling or numbness in hands and feet (neuropathy)
- Increased risk of heart disease, stroke, kidney failure, and blindness

#### Diagnostic Criteria:
- Fasting Plasma Glucose (FPG) ≥126 mg/dL or random plasma glucose ≥200 mg/dL
- H

In [18]:
# Level 10: Advanced Detail for Clinical Research or Specialist Training
prompt = "Provide an in-depth, evidence-based analysis of the symptoms, diagnostic criteria, and treatment approaches for diabetes, including both Type 1 and Type 2. Discuss the pathophysiology, biomarkers, and clinical guidelines, as well as how complications like neuropathy, retinopathy, and cardiovascular disease alter treatment. Include recent clinical research findings, new pharmaceutical options, and the role of lifestyle interventions in managing long-term patient outcomes."
assistant(prompt)

### Human: Provide an in-depth, evidence-based analysis of the symptoms, diagnostic criteria, and treatment approaches for diabetes, including both Type 1 and Type 2. Discuss the pathophysiology, biomarkers, and clinical guidelines, as well as how complications like neuropathy, retinopathy, and cardiovascular disease alter treatment. Include recent clinical research findings, new pharmaceutical options, and the role of lifestyle interventions in managing long-term patient outcomes.
### Assistant : Diabetes is a chronic metabolic disorder characterized by high blood sugar levels due to either insufficient insulin production or insulin resistance. It can be classified into two main types: Type 1 and Type 2.

#### Type 1 Diabetes:
- **Symptoms:** Frequent urination, increased thirst, unexplained weight loss, fatigue, blurred vision, and slow-healing wounds.
- **Diagnostic Criteria:** Presence of autoantibodies (e.g., insulin autoantibodies) and low levels of insulin in the blood.
- **Path